<h2>Basic Analytic Functions Operations</h2>
<p>
This Python Notebook provides some basic workflow operations for the Teradata analytic functions library. It will use the plots, species, and surveys datasets created
when running the PythonBasicAnalytics-Setup SQL notebook.</p>

<i>NOTE: You must have a connection to Teradata Vantage that has the Teradata analytic functions installed.</i>
    


<h3>Get the list of Teradata analytic functions</h3>

In [49]:
help("teradataml")

Help on package teradataml:

NAME
    teradataml

PACKAGE CONTENTS
    analytics (package)
    common (package)
    context (package)
    data (package)
    dataframe (package)
    dbutils (package)
    lib (package)
    options (package)

SUBMODULES
    tdmlctx

DATA
    col_types_dict = {}
    configure = <teradataml.options.configure._Configure object>
    curr_dir = '/opt/conda/lib/python3.6/site-packages/teradataml/data'
    display = <teradataml.options.display._Display object>
    json_data = {}
    td_connection = None
    td_sqlalchemy_engine = None
    temporary_database_name = None
    user_specified_connection = False

FILE
    /opt/conda/lib/python3.6/site-packages/teradataml/__init__.py




<h3>Import statements</h3>

In [1]:
from teradataml.context.context import create_context, remove_context, get_context
from teradataml.dataframe.dataframe import DataFrame
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.options.display import display
import numpy as np

<h3>Create a connection</h3>

In [ ]:
# Replace your cluster details for user, passwd and host
user = "xxxxx"
passwd = "xxxxx"
host = "xxxxx"
eng = create_context(host = host, username = user, password = passwd)
conn = eng.connect()
print(eng)
print(conn)

<h3>Create Teradata DataFrames</h3>

<i>Insert the tables' data using the Teradata SQL notebooks provided (PythonBasicAnalytics-Cleanup.ipynb and PythonBasicAnalytics-Setup.ipynb) before executing the next cell.</i>

In [3]:
plots_data = DataFrame('plots')
species_data = DataFrame('species')
surveys_data = DataFrame('surveys')

In [4]:
plots_data.head(5)

                        plot_type
plot_id                          
3        Long-term Krat Exclosure
5                Rodent Exclosure
4                         Control
2                         Control
1               Spectab exclosure

In [5]:
species_data.head(5)

                       genus          species    taxa
species_id                                           
AS                Ammodramus       savannarum    Bird
CB           Campylorhynchus  brunneicapillus    Bird
BA                   Baiomys          taylori  Rodent
AH          Ammospermophilus          harrisi  Rodent
AB                Amphispiza        bilineata    Bird

In [6]:
surveys_data.head(5)

          month day  year plot_id species_id sex hindfoot_length weight
record_id                                                              
3             7  16  1977       2       DM     F              37   None
5             7  16  1977       3       DM     M              35   None
4             7  16  1977       7       DM     M              36   None
2             7  16  1977       3       NL     M              33   None
1             7  16  1977       2       NL     M              32   None

In [7]:
# Function to verify whether the number of rows of a TeradataML dataframe equals 'value'
def assertLength(df, value):
    try:
        assert(df.shape[0] == value)
    except AssertionError:
        raise

## Verifying the correctness of initial data setup

In [8]:
assertLength(plots_data, 24)
assertLength(species_data, 54)
assertLength(surveys_data, 1236)

In [9]:
sorted_surveys_data = surveys_data.sort('record_id')
sorted_surveys_data.head(10)

          month day  year plot_id species_id sex hindfoot_length weight
record_id                                                              
1             7  16  1977       2       NL     M              32   None
2             7  16  1977       3       NL     M              33   None
3             7  16  1977       2       DM     F              37   None
4             7  16  1977       7       DM     M              36   None
5             7  16  1977       3       DM     M              35   None
6             7  16  1977       1       PF     M              14   None
7             7  16  1977       2       PE     F            None   None
8             7  16  1977       1       DM     M              37   None
9             7  16  1977       1       DM     F              34   None
10            7  16  1977       6       PF     F              20   None

In [10]:
# Asserting some values of the survey data
pandas_data = sorted_surveys_data.to_pandas()

assert(pandas_data.index.name == "record_id")
assert(list(pandas_data.index)[0:10] == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] )
assert(list(pandas_data['plot_id'][0:10]) == [2, 3, 2, 7, 3, 1, 2, 1, 1, 6] )
assert(list(pandas_data['year'][0:10]) == [1977]*10 )
assert(list(pandas_data['month'][0:2]) == [7]*2 )
assert(list(pandas_data['day'][0:10]) == [16]*10 )
assert(list(pandas_data['species_id'][0:10]) == ['NL  ','NL  ','DM  ','DM  ','DM  ','PF  ','PE  ','DM  ','DM  ','PF  '])
assert(list(pandas_data['hindfoot_length'][0:6]) == [32.0, 33.0, 37.0, 36.0, 35.0, 14.0])

## Subsetting, sorting, transformation, joining of Teradata DataFrames

In [11]:
# extract the survey observations for the first three months of 1990 with filter
surveys1990_winter = surveys_data[(surveys_data.year == 1990) & ((surveys_data.month == 1) | 
                                                                 (surveys_data.month == 2) | 
                                                                 (surveys_data.month == 3))]

surveys1990_winter

          month day  year plot_id species_id   sex hindfoot_length weight
record_id                                                                
16924         1   6  1990       6       PF       M              16      7
16964         1   7  1990      13       RM       M              17     11
17229         2  25  1990       8       DM       F              35     40
16882         1   6  1990      23       RM       F              17      9
17180         2  25  1990      13       RM       M              16     10
17027         1  29  1990      21       PG    None            None   None
17292         3  29  1990      23       RM       F              16     12
16884         1   6  1990      24       RM       M              17      9
17149         2  24  1990      17       RM       M              17      9
17189         2  25  1990       8       DM       F              37     41

In [12]:
assertLength(surveys1990_winter, 16)

In [13]:
# Remove redundant 'year' column - as all the rows have same value '1990'
surveys1990_winter = surveys1990_winter.drop(['year'], axis = 1)
surveys1990_winter.head()

          month day plot_id species_id   sex hindfoot_length weight
record_id                                                          
16924         1   6       6       PF       M              16      7
17027         1  29      21       PG    None            None   None
17067         1  30      10       RM       F              18     12
17086         1  30       9       DM       M              38     45
17147         2  24       7       PE       M              22     26
17149         2  24      17       RM       M              17      9
17107         1  30       4       DS       M              50    132
16964         1   7      13       RM       M              17     11
16884         1   6      24       RM       M              17      9
16882         1   6      23       RM       F              17      9

In [14]:
assert('year' not in surveys1990_winter.columns) # Verifying whether 'year' is in list of columns

In [15]:
# sort the 1990 winter surveys data by descending order of species name, 
# then by ascending order of weight. 
sorted_survey1990 = surveys1990_winter.sort(columns=['species_id', 'weight'], ascending=[False, True])
sorted_survey1990

          month day plot_id species_id   sex hindfoot_length weight
record_id                                                          
17170         2  25       3       SH       M              30     80
16884         1   6      24       RM       M              17      9
17149         2  24      17       RM       M              17      9
16882         1   6      23       RM       F              17      9
17180         2  25      13       RM       M              16     10
16964         1   7      13       RM       M              17     11
17292         3  29      23       RM       F              16     12
17067         1  30      10       RM       F              18     12
17027         1  29      21       PG    None            None   None
16924         1   6       6       PF       M              16      7

In [16]:
assert(list(sorted_survey1990.to_pandas().index[0:6]) == [17170, 16884, 17149, 16882, 17180, 16964])

In [17]:
# Count the number of individuals by species observed in the winter of 1990. 
# First define a grouping of our surveys1990_winter teradataml DataFrame with group_by, 
# then call count() to count the rows under each species.

count_1990w = surveys1990_winter.groupby('species_id').count().select(['species_id', 'count_record_id'])
count_1990w = count_1990w.assign(count1 = count_1990w.count_record_id)
count_1990w = count_1990w.drop(['count_record_id'], axis=1)
count_1990w

  species_id count1
0       RM        7
1       OT        1
2       PF        1
3       PE        1
4       SH        1
5       DM        3
6       DS        1
7       PG        1

In [18]:
assertLength(count_1990w, 8)

In [19]:
# Sum of all count1 column values will give total samples (tot_cnt)
total_count = count_1990w.select('count1').sum()
tot_cnt = total_count.to_pandas()['sum_count1'][0]
tot_cnt

16

In [20]:
surveys1990_winter.shape

(16, 8)

In [21]:
# we use the previously obtained count variable to derive the PROPORTION of individuals represented by
# each species, and assign the result to a new 'prop' column.
count_1990w1 = count_1990w.assign(prop = count_1990w.count1 / float(tot_cnt))
count_1990w1

  species_id count1    prop
0       PE        1  0.0625
1       RM        7  0.4375
2       DM        3  0.1875
3       OT        1  0.0625
4       PF        1  0.0625
5       SH        1  0.0625
6       DS        1  0.0625
7       PG        1  0.0625

In [22]:
count_1990w1.shape

(8, 3)

In [23]:
count_1990w1.dtypes # Type of data in every column

species_id      str
count1          int
prop          float

In [24]:
# 0.9999999999999998
assert(round(count_1990w1.select('prop').sum().to_pandas()['sum_prop'][0]) == 1)

In [25]:
# Inner join of DataFrames to get all species data along with count1 and prop.
counts_1990w_join_inner = count_1990w1.join(species_data, on = 'species_id', how = "inner", lsuffix = 't1', rsuffix = 't2')
counts_1990w_join_inner = counts_1990w_join_inner.assign(species_id = counts_1990w_join_inner.t1_species_id)
counts_1990w_join_inner = counts_1990w_join_inner.drop(['t1_species_id', 't2_species_id'], axis = 1)
counts_1990w_join_inner

  count1    prop            genus      species    taxa species_id
0      1  0.0625       Peromyscus     eremicus  Rodent       PE  
1      7  0.4375  Reithrodontomys    megalotis  Rodent       RM  
2      3  0.1875        Dipodomys     merriami  Rodent       DM  
3      1  0.0625      Perognathus       flavus  Rodent       PF  
4      1  0.0625        Onychomys     torridus  Rodent       OT  
5      1  0.0625         Sigmodon     hispidus  Rodent       SH  
6      1  0.0625        Dipodomys  spectabilis  Rodent       DS  
7      1  0.0625        Pooecetes    gramineus    Bird       PG  

In [26]:
counts_1990w_join_inner.shape

(8, 6)

In [27]:
counts_1990w_join_inner.to_pandas()

,count1,prop,genus,species,taxa,species_id
0,1,0.0625,Perognathus,flavus,Rodent,PF
1,1,0.0625,Pooecetes,gramineus,Bird,PG
2,1,0.0625,Dipodomys,spectabilis,Rodent,DS
3,1,0.0625,Peromyscus,eremicus,Rodent,PE
4,1,0.0625,Sigmodon,hispidus,Rodent,SH
5,1,0.0625,Onychomys,torridus,Rodent,OT
6,3,0.1875,Dipodomys,merriami,Rodent,DM
7,7,0.4375,Reithrodontomys,megalotis,Rodent,RM


In [28]:
# Asserting the columns in counts_1990w_join_inner
expected_columns = list(set(species_data.columns + count_1990w1.columns))
assert(len(counts_1990w_join_inner.columns) == len(expected_columns))
assert(all(col in expected_columns for col in counts_1990w_join_inner.columns))

In [29]:
display.max_rows = 20

In [30]:
print(count_1990w1.shape)
print(count_1990w1.columns)

(8, 3)
['species_id', 'count1', 'prop']


In [31]:
print(species_data.shape)
print(species_data.columns)

(54, 4)
['species_id', 'genus', 'species', 'taxa']


In [32]:
list(set(species_data.columns + count_1990w1.columns))

['genus', 'species_id', 'species', 'taxa', 'prop', 'count1']

In [33]:
# Left join of DataFrames to get all species data along with count1 and prop.
counts_1990w_join_left = count_1990w1.join(species_data, on = 'species_id', how = "left", lsuffix = 't1', rsuffix = 't2')
counts_1990w_join_left = counts_1990w_join_left.assign(species_id = counts_1990w_join_left.t1_species_id)
counts_1990w_join_left = counts_1990w_join_left.drop(['t1_species_id', 't2_species_id'], axis = 1)
print(counts_1990w_join_left.shape)
counts_1990w_join_left # Observe one species with " " species_id - which is not present in counts_1990w_join_inner

(8, 6)


  count1    prop            genus      species    taxa species_id
0      1  0.0625      Perognathus       flavus  Rodent       PF  
1      1  0.0625        Pooecetes    gramineus    Bird       PG  
2      1  0.0625        Dipodomys  spectabilis  Rodent       DS  
3      1  0.0625       Peromyscus     eremicus  Rodent       PE  
4      1  0.0625         Sigmodon     hispidus  Rodent       SH  
5      1  0.0625        Onychomys     torridus  Rodent       OT  
6      3  0.1875        Dipodomys     merriami  Rodent       DM  
7      7  0.4375  Reithrodontomys    megalotis  Rodent       RM  

In [34]:
counts_1990w_join_left.sum().to_pandas() # sum_count1 should be 16 (total samples) and sum_prop should be 1 (adding up all proportions)

,sum_count1,sum_prop
0,16,1.0


In [35]:
assert(counts_1990w_join_left.sum().to_pandas()['sum_count1'][0] == 16)
assert(counts_1990w_join_left.sum().to_pandas()['sum_prop'][0] == 1.0)

In [36]:
list(counts_1990w_join_left.select('genus').to_pandas()['genus'])

['Peromyscus',
 'Reithrodontomys',
 'Dipodomys',
 'Perognathus',
 'Onychomys',
 'Sigmodon',
 'Dipodomys',
 'Pooecetes']

In [37]:
# Get max value of every column grouped by genus
counts_1990w_join_left.groupby('genus').max()

             genus max_count1  max_prop  max_species max_taxa max_species_id
0        Onychomys          1    0.0625     torridus   Rodent           OT  
1       Peromyscus          1    0.0625     eremicus   Rodent           PE  
2      Perognathus          1    0.0625       flavus   Rodent           PF  
3        Pooecetes          1    0.0625    gramineus     Bird           PG  
4         Sigmodon          1    0.0625     hispidus   Rodent           SH  
5        Dipodomys          3    0.1875  spectabilis   Rodent           DS  
6  Reithrodontomys          7    0.4375    megalotis   Rodent           RM  

In [38]:
assertLength(counts_1990w_join_left.groupby('genus').max(), 7)

In [39]:
# Getting count of samples under each taxa
taxa_grouby_sum = counts_1990w_join_left.groupby('taxa').sum().select(['taxa', 'sum_count1'])
pd_taxa_groupby_sum = taxa_grouby_sum.to_pandas()
pd_taxa_groupby_sum

,taxa,sum_count1
0,Bird,1
1,Rodent,15


# Calculate the fraction of total counts by taxa (birds or rodents) represented by each species within that taxon.

In [40]:
taxa_groupby_join_counts = counts_1990w_join_left.join(taxa_grouby_sum, how = 'right', on = 'taxa', 
                                       lsuffix= 'cnt', rsuffix= 'x')
taxa_groupby_join_counts

  count1    prop            genus      species cnt_taxa  x_taxa species_id sum_count1
0      3  0.1875        Dipodomys     merriami   Rodent  Rodent       DM           15
1      1  0.0625        Onychomys     torridus   Rodent  Rodent       OT           15
2      1  0.0625        Dipodomys  spectabilis   Rodent  Rodent       DS           15
3      1  0.0625       Peromyscus     eremicus   Rodent  Rodent       PE           15
4      1  0.0625         Sigmodon     hispidus   Rodent  Rodent       SH           15
5      1  0.0625      Perognathus       flavus   Rodent  Rodent       PF           15
6      7  0.4375  Reithrodontomys    megalotis   Rodent  Rodent       RM           15
7      1  0.0625        Pooecetes    gramineus     Bird    Bird       PG            1

In [41]:
taxa_groupby_join_counts = taxa_groupby_join_counts.drop(['cnt_taxa'], axis=1) # Dropping repeated column
taxa_groupby_join_counts 

  count1    prop            genus      species  x_taxa species_id sum_count1
0      1  0.0625         Sigmodon     hispidus  Rodent       SH           15
1      3  0.1875        Dipodomys     merriami  Rodent       DM           15
2      1  0.0625      Perognathus       flavus  Rodent       PF           15
3      1  0.0625        Onychomys     torridus  Rodent       OT           15
4      1  0.0625        Pooecetes    gramineus    Bird       PG            1
5      7  0.4375  Reithrodontomys    megalotis  Rodent       RM           15
6      1  0.0625       Peromyscus     eremicus  Rodent       PE           15
7      1  0.0625        Dipodomys  spectabilis  Rodent       DS           15

In [42]:
taxa_groupby_prop_counts = taxa_groupby_join_counts.assign(
                                    prop = taxa_groupby_join_counts.count1 / taxa_groupby_join_counts.sum_count1)
# 'prop' column will have zeros as of now as every value lies between 0 and 1
taxa_groupby_prop_counts

  count1 prop            genus      species  x_taxa species_id sum_count1
0      3    0        Dipodomys     merriami  Rodent       DM           15
1      1    0        Onychomys     torridus  Rodent       OT           15
2      1    0        Dipodomys  spectabilis  Rodent       DS           15
3      1    0       Peromyscus     eremicus  Rodent       PE           15
4      1    0         Sigmodon     hispidus  Rodent       SH           15
5      1    0      Perognathus       flavus  Rodent       PF           15
6      7    0  Reithrodontomys    megalotis  Rodent       RM           15
7      1    1        Pooecetes    gramineus    Bird       PG            1

In [43]:
# The features of all surveyed individuals of Reithrodontomys montanus (RO).

surveys_RO = surveys_data[surveys_data.species_id == 'RO']
surveys_RO

Empty DataFrame
Columns: [month, day, year, plot_id, species_id, sex, hindfoot_length, weight]
Index: []

In [ ]:
# Find the average weight and hindfoot length of Dipodomys merriami (DM) individuals 
# observed in each month (irrespective of the year). Make sure to exclude NA values.

In [44]:
surveys_dm = surveys_data[surveys_data.species_id == 'DM']
surveys_dm

          month day  year plot_id species_id sex hindfoot_length weight
record_id                                                              
5             7  16  1977       3       DM     M              35   None
33871         4  17  2002       4       DM     M              36     52
20657         2  19  1993      17       DM     F              32     40
18108        12  15  1990      12       DM     F              36     41
12072        12  14  1986      24       DM     F              36     43
938           6   8  1978      20       DM     M              38     46
31322         8  26  2000       9       DM     M              36     51
9             7  16  1977       1       DM     F              34   None
24153         6  14  1996       4       DM     M              38     46
4434          5   4  1981       9       DM     F              36     44

In [45]:
surveys_dm.to_pandas().shape

(360, 8)

In [46]:
survey_monthwise = surveys_dm.groupby('month').agg({'weight' : 'mean', 'hindfoot_length' : 'mean'})

In [47]:
assertLength(survey_monthwise, 12)

In [48]:
expected_columns = ['month', 'mean_weight', 'mean_hindfoot_length']
assert(len(survey_monthwise.columns) == len(expected_columns))
assert(all(col in expected_columns for col in survey_monthwise.columns))

<span style="font-size:16px;">For more information on the Teradata analytic functions, refer to the [Teradata Documentation](https://docs.teradata.com/) and search for Teradata Python Package.</span>

Copyright 2019 Teradata. All rights reserved.